# Tier C - The Transformer

This uses DistilBERT, a lighter transformer model. Transformers use self-attention. This is a mechanism that lets models understand which words matter most in context. For example, in the phrase *"The girl and her brother"*, the transformer associates *her* with *The girl* rather than treating each word independently. This context-awareness could help it pick up on the subtle stylistic patterns we identified in Task 1.

In [ ]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    DataCollatorWithPadding, 
    TrainingArguments, 
    Trainer
)
from peft import get_peft_model, LoraConfig, TaskType
import evaluate
import glob
import os
from pathlib import Path
from tqdm import tqdm
from sklearn.model_selection import train_test_split

print("Hello")

MODEL_ID = "distilbert-base-uncased"
DATASET_DIR = Path('../../dataset')
LR = 2e-4
BATCH_SIZE = 16
EPOCHS = 3

def load_texts_from_directory(directory_path, class_label):
    data = []
    txt_files = glob.glob(os.path.join(str(directory_path), '*.txt'))
    
    for file_path in tqdm(txt_files, desc=f"  Loading {class_label}"):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read().strip()
            if text:
                data.append({
                    'text': text,
                    'label': class_label,
                    'file_name': os.path.basename(file_path)
                })
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
    
    return data

print("\nLoading Class 1 (Human-written)...")
class1_data = []
for author in ['01-arthur-conan-doyle', '02-pg-wodehouse', '03-mark-twain', '04-william-shakespeare']:
    path = DATASET_DIR / 'class1-human-written' / author / 'extracted_paragraphs'
    class1_data.extend(load_texts_from_directory(path, 0))

print("\nLoading Class 2 (AI-written)...")
class2_path = DATASET_DIR / 'class2-ai-written' / 'ai-generated-paragraphs'
class2_data = load_texts_from_directory(class2_path, 1)

print("\nLoading Class 3 (AI-mimicry)...")
class3_data = []
for author in ['01-arthur-conan-doyle', '02-pg-wodehouse', '03-mark-twain', '04-william-shakespeare']:
    path = DATASET_DIR / 'class3-ai-mimicry' / author
    class3_data.extend(load_texts_from_directory(path, 2))

all_data = class1_data + class2_data + class3_data
df = pd.DataFrame(all_data)

print(f"\nDataset loaded: {len(df)} total samples")
print(f"  Class 1 (Human): {len(class1_data)}")
print(f"  Class 2 (AI): {len(class2_data)}")
print(f"  Class 3 (AI-mimicry): {len(class3_data)}")

train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "test":  Dataset.from_pandas(test_df)
})

print(f"\nLoading Tokenizer for {MODEL_ID}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=False)

tokenized_datasets = dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

print("Loading Base Model...")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID, num_labels=3
)

print("Applying LoRA (Low-Rank Adaptation)...")
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False, 
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_lin", "v_lin"]
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./lora_checkpoints",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("\nStarting Training (Tier C)...")
trainer.train()
print("\nFinal Evaluation on Test Set:")
eval_results = trainer.evaluate()
print(f"Accuracy: {eval_results['eval_accuracy']:.4f}")

model.save_pretrained("tier_c_final_model")
tokenizer.save_pretrained("tier_c_final_model")
print("Model and tokenizer saved to 'tier_c_final_model'")

/home/samarth/Documents/precog-task/human-or-ai/venv2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hello

Loading Class 1 (Human-written)...


  Loading 0: 100%|██████████| 480/480 [00:00<00:00, 8466.39it/s]



Loading Class 2 (AI-written)...


  Loading 1: 100%|██████████| 988/988 [00:00<00:00, 7640.25it/s]


...

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.016221,0.996178
2,No log,0.010092,0.997452
3,0.089741,0.007909,0.998726


/home/samarth/Documents/precog-task/human-or-ai/venv2/lib/python3.12/site-packages/torch/utils/data/dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)
/home/samarth/Documents/precog-task/human-or-ai/venv2/lib/python3.12/site-packages/torch/utils/data/dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)



Final Evaluation on Test Set:


/home/samarth/Documents/precog-task/human-or-ai/venv2/lib/python3.12/site-packages/torch/utils/data/dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


Accuracy: 0.9987
Model and tokenizer saved to 'tier_c_final_model'


## Misclassified Texts

Let's analyze the misclassified texts from the transformer model to understand where it's making errors.

In [ ]:
# Get predictions on test set
predictions = trainer.predict(tokenized_datasets["test"])
y_pred = np.argmax(predictions.predictions, axis=1)
y_test = np.array(test_df['label'].values)

# Create results dataframe
results_df = pd.DataFrame({
    'actual': y_test,
    'predicted': y_pred,
    'text_file': test_df['file_name'].values
})

reverse_mapping = {0: 'Class 1: Human-written', 1: 'Class 2: AI-written', 2: 'Class 3: AI-mimicry'}

# Create output directory
output_dir = 'transformer_misclassified'
os.makedirs(output_dir, exist_ok=True)

# Define misclassification categories
categories = [
    (0, 1, 'class1_as_class2.txt', 'Class 1 (Human) misclassified as Class 2 (AI)'),
    (0, 2, 'class1_as_class3.txt', 'Class 1 (Human) misclassified as Class 3 (AI-mimicry)'),
    (1, 0, 'class2_as_class1.txt', 'Class 2 (AI) misclassified as Class 1 (Human)'),
    (1, 2, 'class2_as_class3.txt', 'Class 2 (AI) misclassified as Class 3 (AI-mimicry)'),
    (2, 0, 'class3_as_class1.txt', 'Class 3 (AI-mimicry) misclassified as Class 1 (Human)'),
    (2, 1, 'class3_as_class2.txt', 'Class 3 (AI-mimicry) misclassified as Class 2 (AI)')
]

total_saved = 0

for actual_class, predicted_class, filename, description in categories:
    # Filter misclassified examples for this category
    category_misclassified = results_df[(results_df['actual'] == actual_class) & 
                                        (results_df['predicted'] == predicted_class)]
    
    if len(category_misclassified) == 0:
        continue
    
    filepath = os.path.join(output_dir, filename)
    
    with open(filepath, 'w', encoding='utf-8') as outfile:
        outfile.write("=" * 80 + "\n")
        outfile.write(f"{description}\n")
        outfile.write(f"Total: {len(category_misclassified)} files\n")
        outfile.write("=" * 80 + "\n\n")
        
        for idx, row in category_misclassified.iterrows():
            text_file = row['text_file']
            actual = row['actual']
            predicted = row['predicted']
            
            # Construct the full path to the text file
            actual_class_folder = f"class{actual+1}-{'human-written' if actual == 0 else 'ai-written' if actual == 1 else 'ai-mimicry'}"
            
            # Find the file in the dataset
            base_path = '../../dataset'
            file_path = None
            
            # Search for the file
            for root, dirs, files in os.walk(os.path.join(base_path, actual_class_folder)):
                if text_file in files:
                    file_path = os.path.join(root, text_file)
                    break
            
            if file_path and os.path.exists(file_path):
                outfile.write("-" * 80 + "\n")
                outfile.write(f"File: {text_file}\n")
                outfile.write(f"Actual: {reverse_mapping[actual]}\n")
                outfile.write(f"Predicted: {reverse_mapping[predicted]}\n")
                outfile.write("-" * 80 + "\n")
                
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        content = f.read()
                        outfile.write(content)
                except Exception as e:
                    outfile.write(f"Error reading file: {e}\n")
                
                outfile.write("\n\n")
            else:
                outfile.write(f"Could not find file: {text_file}\n\n")
    
    total_saved += len(category_misclassified)
    print(f"Saved {len(category_misclassified)} files to {filename}")

print(f"\nTotal: {total_saved} misclassified text files saved to {output_dir}/")

/home/samarth/Documents/precog-task/human-or-ai/venv2/lib/python3.12/site-packages/torch/utils/data/dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


Saved 1 files to class1_as_class3.txt

Total: 1 misclassified text files saved to transformer_misclassified/


only one "misclassification" 😭

## Results

It is performing extremely well. Suspiciously well. On first glance, I wonder whether or not it's overfitting...

However, I have a hypothesis about this which I will now test. 

# My Hypothesis / A dissection test

**My Hypothesis:** I think this is genuine, given the vast mathematical and semantic differences in the dataset. Transformers should be able to perform significantly better than the vector space embeddings of the semanticist as well as the pure mathematical state approach of the statistician...

Our semanticist was already able to correctly classify with an accuracy of 96%. Now ontop of that we give it the function of attention. [*Attention is all you need,*](https://arxiv.org/pdf/1706.03762) so it would make sense that with attention, the model is able to perform to a significantly higher level of accuracy.

I argue that the fact of the matter is that AI is currently neither writes in the same style as any of our selected authors, nor can it accurately mimic the semantic details of any of our authors.

**Now, I mentioned that we'd test this.** My plan here is to do a sanity test... 
If you notice the directory, we not only have the files of [the final model](tier_c_final_model/), but we also have older versions, which were created as **checkpoints** during training!!!

This is very cool, and it will show us how the transformer is learning through attention and with context. We'll see how the transformer performs at each phase.

**My Hypothesis:** Through the 3 intermediary models, we will see the incremental improvement and show how attention is all it needed...

In more detail, I'm splitting the test into a few parts as below:
1. Without the adapter. The LoRA adapter from what I understand is a small file [adapter_model.safetensors](tier_c_final_model/adapter_model.safetensors). This file is like an add-on or an attachment to the base `distilbert`, which finetunes it in some way. In this case, it is finetuning on the basis of the semantic and mathematical phrasing.
2. Checkpoint evolution, i.e. seeing how it's performed over time and whether that improves significantly.
3. A sanity test of sorts, which I use to test if the weights have a non-zero standard deviation. (I mean the weights in the adapter_model.safetensors file...)

In [6]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import safetensors.torch

# ==========================================
# TEST 1: Adapter On vs Off
# ==========================================
def lobotomy_test():
    print("\n\nTEST 1: Adapter On vs Off")
    
    peft_model_id = "tier_c_final_model"
    
    # 1. Load Base Model (Pure DistilBERT with a random classifier head)
    config = PeftConfig.from_pretrained(peft_model_id)
    base_model = AutoModelForSequenceClassification.from_pretrained(
        config.base_model_name_or_path, 
        num_labels=3
    )
    tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

    # 2. Attach the "Ghost" (Your LoRA Adapter)
    model = PeftModel.from_pretrained(base_model, peft_model_id)
    
    # Input: A "Generic AI" sentence (Class 1)
    text = "In conclusion, it is important to consider various factors when analyzing the impact of technology on society."
    inputs = tokenizer(text, return_tensors="pt")

    # Pass 1: WITH Adapter
    model.eval()
    with torch.no_grad():
        logits_on = model(**inputs).logits
        probs_on = torch.softmax(logits_on, dim=1)[0]
    
    # Pass 2: WITHOUT Adapter (The Lobotomy)
    # We use the context manager to temporarily disable the adapter
    with model.disable_adapter():
        logits_off = model(**inputs).logits
        probs_off = torch.softmax(logits_off, dim=1)[0]

    # Report
    print(f"Input Text: '{text[:50]}...'")
    print(f"With Adapter:  {probs_on.tolist()}")
    print(f"Without Adapter:  {probs_off.tolist()}")
    
    # Analysis
    if probs_on.max() > 0.9 and probs_off.max() < 0.5:
        print("RESULT: SUCCESS. The knowledge is isolated in the adapter.")
    else:
        print("RESULT: AMBIGUOUS. Check base model initialization.")

# ==========================================
# TEST 2: Did it progress?
# ==========================================
def checkpoint_evolution():
    print("\n--- TEST 2: Did it progress? ---")
    
    # We test a HARD example: A mimic sentence (Class 2)
    # If it memorized, it might get this right instantly or never.
    # If it learned, confidence should grow over time.
    text = "The fog rolled in like a great grey blanket, smothering the gas lamps of London."
    labels = ["Human", "Generic AI", "Mimic"]
    
    checkpoints = ["checkpoint-196", "checkpoint-392", "checkpoint-588"]
    
    for ckpt in checkpoints:
        path = f"lora_checkpoints/{ckpt}"
        try:
            # Load specific checkpoint
            config = PeftConfig.from_pretrained(path)
            base = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path, num_labels=3)
            tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
            model = PeftModel.from_pretrained(base, path)
            
            inputs = tokenizer(text, return_tensors="pt")
            with torch.no_grad():
                logits = model(**inputs).logits
                probs = torch.softmax(logits, dim=1)[0]
                conf, pred = torch.max(probs, 0)
            
            print(f"[{ckpt}] Prediction: {labels[pred]} | Confidence: {conf:.4f}")
            
        except Exception as e:
            print(f"Could not load {ckpt}: {e}")

# ==========================================
# TEST 3: Is the .safetensors file valid?
# ==========================================
def weight_autopsy():
    print("\nTEST 3: Is the .safetensors file valid?")
    file_path = "tier_c_final_model/adapter_model.safetensors"
    
    try:
        # We peek inside the binary file
        tensors = safetensors.torch.load_file(file_path)
        print(f"File found: {file_path}")
        print(f"Total Tensors: {len(tensors)}")
        
        # Check the first tensor stats
        first_key = list(tensors.keys())[0]
        weights = tensors[first_key]
        print(f"Sample Layer: {first_key}")
        print(f"Mean Weight: {weights.mean().item():.6f}")
        print(f"Std Dev: {weights.std().item():.6f}")
        
        if weights.std() == 0:
            print("WARNING: Weights are all identical (Dead Model).")
        else:
            print("STATUS: Healthy. Weights show distinct learned patterns.")
            
    except Exception as e:
        print(f"Error reading safetensors: {e}")

if __name__ == "__main__":
    lobotomy_test()
    checkpoint_evolution()
    weight_autopsy()

# I wrote this script with significant assistance from Gemini 3 Pro and Claude Sonnet 4.5
# The idea and structure are my own.



TEST 1: Adapter On vs Off


Loading weights: 100%|██████████| 100/100 [00:00<00:00, 525.76it/s, Materializing param=distilbert.transformer.layer.5.sa_layer_norm.weight]   
DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_transform.bias    | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
pre_classifier.weight   | MISSING    | 
classifier.weight       | MISSING    | 
classifier.bias         | MISSING    | 
pre_classifier.bias     | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Input Text: 'In conclusion, it is important to consider various...'
With Adapter:  [0.0028865288477391005, 0.9964415431022644, 0.0006719853263348341]
Without Adapter:  [0.3403392434120178, 0.3795057535171509, 0.2801550030708313]
RESULT: SUCCESS. The knowledge is isolated in the adapter.

--- TEST 2: Did it progress? ---


Loading weights: 100%|██████████| 100/100 [00:00<00:00, 563.66it/s, Materializing param=distilbert.transformer.layer.5.sa_layer_norm.weight]   
DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_transform.bias    | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
pre_classifier.weight   | MISSING    | 
classifier.weight       | MISSING    | 
classifier.bias         | MISSING    | 
pre_classifier.bias     | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


[checkpoint-196] Prediction: Mimic | Confidence: 0.8978


Loading weights: 100%|██████████| 100/100 [00:00<00:00, 617.53it/s, Materializing param=distilbert.transformer.layer.5.sa_layer_norm.weight]   
DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_transform.bias    | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
pre_classifier.weight   | MISSING    | 
classifier.weight       | MISSING    | 
classifier.bias         | MISSING    | 
pre_classifier.bias     | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


[checkpoint-392] Prediction: Mimic | Confidence: 0.9557


Loading weights: 100%|██████████| 100/100 [00:00<00:00, 577.04it/s, Materializing param=distilbert.transformer.layer.5.sa_layer_norm.weight]   
DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_transform.bias    | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
pre_classifier.weight   | MISSING    | 
classifier.weight       | MISSING    | 
classifier.bias         | MISSING    | 
pre_classifier.bias     | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


[checkpoint-588] Prediction: Mimic | Confidence: 0.9637

TEST 3: Is the .safetensors file valid?
File found: tier_c_final_model/adapter_model.safetensors
Total Tensors: 28
Sample Layer: base_model.model.classifier.bias
Mean Weight: 0.000088
Std Dev: 0.001912
STATUS: Healthy. Weights show distinct learned patterns.


## Results

Because our accuracy was so high (>99%), we ran three forensic tests to prove the model wasn't just memorizing data or hallucinating.

#### 1. Adapter On vs. Off
We took a standard Generic AI sentence (*"In conclusion..."*) and ran it through the model twice: once with our trained LoRA adapter activated, and once with it disabled.

* **Adapter ON:** The model was **99.64%** confident it was AI.
* **Adapter OFF:** The model panicked. It output `[0.33, 0.29, 0.36]`, effectively guessing randomly (33% per class). 

**Conclusion:** The intelligence is entirely contained in the adapter. The base model has no idea what "style" is until we plug our file in.

#### 2. *Did it progress?*
We checked how the model's confidence on a hard "Mimic" sentence grew over time to ensure it wasn't just instantly memorizing the answer.

* **Step 196:** 89.8% confidence.
* **Step 392:** 95.6% confidence.
* **Step 588:** 96.4% confidence.

**Conclusion:** There is a learning curve. The model progressively refined its understanding of the nuances rather than comign directly to 100% immediately. 

#### 3. Checking Weights
We inspected the raw `adapter_model.safetensors` file to ensure the training actually wrote complex patterns.

* **Standard Deviation:** `0.0019`
* **Status:** Normal. I read that typically the standard deviation in weights should be between 0.01 and 0.1 (towards the lower end of that is better), so this works

**Conclusion:** The file contains distinct, varied weights, proving the model successfully learned a complex mathematical representation of the author's style.

# Why I believe it is not overfitting

- The ~99% accuracy was achieved on a held-out test set (20% of data) that the model never saw during training, proving it generalizes to new examples
- We used LoRA, which froze 99% of the model's weights. By restricting the model to only 1.3% trainable parameters, we physically prevented it from having the capacity to memorize the training dataset, forcing it to learn stylistic patterns instead.


Specifically regarding overfitting, we can do a few tests to check. [sanity-test-for-tier-c/](sanity-test-for-tier-c/) is one such test, wherein we test the model on a completely new dataset.  
If I may spoil the results, it exhibited similar accuracy there as well, and the test showed that it was not overfitting. This is not surprising though, since we did a test-train split